문자단위 RNN 언어모델


In [100]:
#기초 라이브러리 임포트
import numpy as np
import urllib.request
from tensorflow.keras.utils import to_categorical


데이터 불러오기 

이상한나라의 엘리스 


In [101]:
# urllib 패키지는 외부 url 을 이용해 인터넷 리소스를 가지고 올 수 있음

# 데이터 로드 
urllib.request.urlretrieve("http://www.gutenberg.org/files/11/11-0.txt",filename ="11-0.txt")



('11-0.txt', <http.client.HTTPMessage at 0x7ff6f0689eb0>)

In [102]:
f = open('11-0.txt', 'rb')
sentences = []
for sentence in f: # 데이터로부터 한 줄씩 읽는다.
    sentence = sentence.strip() # strip()을 통해 \r, \n을 제거한다.
    sentence = sentence.lower() # 소문자화.
    sentence = sentence.decode('ascii', 'ignore') # \xe2\x80\x99 등과 같은 바이트 열 제거
    if len(sentence) > 0:
        sentences.append(sentence)
f.close()


In [103]:
sentences[:5]

['the project gutenberg ebook of alices adventures in wonderland, by lewis carroll',
 'this ebook is for the use of anyone anywhere in the united states and',
 'most other parts of the world at no cost and with almost no restrictions',
 'whatsoever. you may copy it, give it away or re-use it under the terms',
 'of the project gutenberg license included with this ebook or online at']

In [104]:
total_data = ' '.join(sentences)
print('문자열의 길이 또는 총 문자의 개수: %d' % len(total_data))

문자열의 길이 또는 총 문자의 개수: 159484


In [105]:
print(total_data[:200])

the project gutenberg ebook of alices adventures in wonderland, by lewis carroll this ebook is for the use of anyone anywhere in the united states and most other parts of the world at no cost and with


In [106]:
# 중복을 제거한 문자 집합
char_vocab = sorted(list(set(total_data)))

vocab_size = len(char_vocab)
print("문자 집합의 크기 ", vocab_size) 

문자 집합의 크기  56


In [107]:
char_to_index = dict((char,index) for index, char in enumerate(char_vocab))

print(char_to_index)

{' ': 0, '!': 1, '"': 2, '#': 3, '$': 4, '%': 5, "'": 6, '(': 7, ')': 8, '*': 9, ',': 10, '-': 11, '.': 12, '/': 13, '0': 14, '1': 15, '2': 16, '3': 17, '4': 18, '5': 19, '6': 20, '7': 21, '8': 22, '9': 23, ':': 24, ';': 25, '?': 26, '[': 27, ']': 28, '_': 29, 'a': 30, 'b': 31, 'c': 32, 'd': 33, 'e': 34, 'f': 35, 'g': 36, 'h': 37, 'i': 38, 'j': 39, 'k': 40, 'l': 41, 'm': 42, 'n': 43, 'o': 44, 'p': 45, 'q': 46, 'r': 47, 's': 48, 't': 49, 'u': 50, 'v': 51, 'w': 52, 'x': 53, 'y': 54, 'z': 55}


In [108]:
index_to_char = {}
for key, value in char_to_index.items():
    index_to_char[value] = key

In [109]:
train_X = 'appl'

train_y = 'pple'

In [110]:
seq_length = 60

# 문자열의 길이를 seq_length로 나누면 전처리 후 생겨날 샘플 수
n_samples = int(np.floor((len(total_data) - 1) / seq_length))
print ('샘플의 수 : {}'.format(n_samples))

샘플의 수 : 2658


In [111]:
train_X = []
train_y = []

for i in range(n_samples):
    # 0:60 -> 60:120 -> 120:180로 loop를 돌면서 문장 샘플을 1개씩 pick.
    X_sample = total_data[i * seq_length: (i + 1) * seq_length]

    # 정수 인코딩
    X_encoded = [char_to_index[c] for c in X_sample]
    train_X.append(X_encoded)

    # 오른쪽으로 1칸 쉬프트
    y_sample = total_data[i * seq_length + 1: (i + 1) * seq_length + 1]
    y_encoded = [char_to_index[c] for c in y_sample]
    train_y.append(y_encoded)

In [112]:
print('X 데이터의 첫번째 샘플 :',train_X[0])
print('y 데이터의 첫번째 샘플 :',train_y[0])
print('-'*50)
print('X 데이터의 첫번째 샘플 디코딩 :',[index_to_char[i] for i in train_X[0]])
print('y 데이터의 첫번째 샘플 디코딩 :',[index_to_char[i] for i in train_y[0]])

X 데이터의 첫번째 샘플 : [49, 37, 34, 0, 45, 47, 44, 39, 34, 32, 49, 0, 36, 50, 49, 34, 43, 31, 34, 47, 36, 0, 34, 31, 44, 44, 40, 0, 44, 35, 0, 30, 41, 38, 32, 34, 48, 0, 30, 33, 51, 34, 43, 49, 50, 47, 34, 48, 0, 38, 43, 0, 52, 44, 43, 33, 34, 47, 41, 30]
y 데이터의 첫번째 샘플 : [37, 34, 0, 45, 47, 44, 39, 34, 32, 49, 0, 36, 50, 49, 34, 43, 31, 34, 47, 36, 0, 34, 31, 44, 44, 40, 0, 44, 35, 0, 30, 41, 38, 32, 34, 48, 0, 30, 33, 51, 34, 43, 49, 50, 47, 34, 48, 0, 38, 43, 0, 52, 44, 43, 33, 34, 47, 41, 30, 43]
--------------------------------------------------
X 데이터의 첫번째 샘플 디코딩 : ['t', 'h', 'e', ' ', 'p', 'r', 'o', 'j', 'e', 'c', 't', ' ', 'g', 'u', 't', 'e', 'n', 'b', 'e', 'r', 'g', ' ', 'e', 'b', 'o', 'o', 'k', ' ', 'o', 'f', ' ', 'a', 'l', 'i', 'c', 'e', 's', ' ', 'a', 'd', 'v', 'e', 'n', 't', 'u', 'r', 'e', 's', ' ', 'i', 'n', ' ', 'w', 'o', 'n', 'd', 'e', 'r', 'l', 'a']
y 데이터의 첫번째 샘플 디코딩 : ['h', 'e', ' ', 'p', 'r', 'o', 'j', 'e', 'c', 't', ' ', 'g', 'u', 't', 'e', 'n', 'b', 'e', 'r', 'g', ' ', 'e',

In [113]:
print(len(train_X[1]))

60


In [114]:
# 원핫 인코딩 진행
train_X = to_categorical(train_X)
train_y = to_categorical(train_y)


In [115]:
print('train_X의 크기(shape) : {}'.format(train_X.shape)) # 원-핫 인코딩
print('train_y의 크기(shape) : {}'.format(train_y.shape)) # 원-핫 인코딩

print(train_X.shape[2])

train_X의 크기(shape) : (2658, 60, 56)
train_y의 크기(shape) : (2658, 60, 56)
56


In [117]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, TimeDistributed

hidden_units = 256

model = Sequential()
model.add(LSTM(hidden_units, input_shape=(None, 56), return_sequences=True))
model.add(LSTM(hidden_units, return_sequences=True))
model.add(TimeDistributed(Dense(vocab_size, activation='softmax')))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(train_X, train_y, epochs=20, verbose=2)


Epoch 1/20
84/84 - 15s - loss: 3.0736 - accuracy: 0.1823 - 15s/epoch - 178ms/step
Epoch 2/20
84/84 - 12s - loss: 2.7688 - accuracy: 0.2382 - 12s/epoch - 147ms/step
Epoch 3/20
84/84 - 13s - loss: 2.4168 - accuracy: 0.3219 - 13s/epoch - 152ms/step
Epoch 4/20
84/84 - 12s - loss: 2.2779 - accuracy: 0.3540 - 12s/epoch - 146ms/step
Epoch 5/20
84/84 - 12s - loss: 2.1823 - accuracy: 0.3772 - 12s/epoch - 146ms/step
Epoch 6/20
84/84 - 12s - loss: 2.1061 - accuracy: 0.3955 - 12s/epoch - 147ms/step
Epoch 7/20
84/84 - 12s - loss: 2.0420 - accuracy: 0.4124 - 12s/epoch - 144ms/step
Epoch 8/20
84/84 - 12s - loss: 1.9865 - accuracy: 0.4251 - 12s/epoch - 145ms/step
Epoch 9/20
84/84 - 12s - loss: 1.9379 - accuracy: 0.4382 - 12s/epoch - 148ms/step
Epoch 10/20
84/84 - 13s - loss: 1.8900 - accuracy: 0.4510 - 13s/epoch - 157ms/step
Epoch 11/20
84/84 - 13s - loss: 1.8483 - accuracy: 0.4635 - 13s/epoch - 149ms/step
Epoch 12/20
84/84 - 12s - loss: 1.8092 - accuracy: 0.4739 - 12s/epoch - 146ms/step
Epoch 13/20
8

In [118]:
def sentence_generation(model, length):
    # 문자에 대한 랜덤한 정수 생성
    ix = [np.random.randint(vocab_size)]

    # 랜덤한 정수로부터 맵핑되는 문자 생성
    y_char = [index_to_char[ix[-1]]]
    print(ix[-1],'번 문자',y_char[-1],'로 예측을 시작!')

    # (1, length, 55) 크기의 X 생성. 즉, LSTM의 입력 시퀀스 생성
    X = np.zeros((1, length, vocab_size))

    for i in range(length):
        # X[0][i][예측한 문자의 인덱스] = 1, 즉, 예측 문자를 다음 입력 시퀀스에 추가
        X[0][i][ix[-1]] = 1
        print(index_to_char[ix[-1]], end="")
        ix = np.argmax(model.predict(X[:, :i+1, :])[0], 1)
        y_char.append(index_to_char[ix[-1]])
    return ('').join(y_char)

In [119]:
result = sentence_generation("u",model, 100)
print(result)

KeyError: 'u'